In [13]:
import random, os, time
import matplotlib.pyplot as plt
import numpy as np
from ctypes import *
from mpl_toolkits.mplot3d import Axes3D

%run "~\Documents\GitHub\ProjetAnnuel3IBD\projet\python\encapsulate\encapsulateSharedMethods.py"
%run "~\Documents\GitHub\ProjetAnnuel3IBD\projet\python\encapsulate\encapsulateRBF.py"
pathDLL = "C:/Users/nico_/Documents/GitHub/ProjetAnnuel3IBD/projet/MLAlgorithms/ML_Library/x64/Release/ML_Library.dll"

myDll = CDLL(pathDLL)

pathSave = "C:/Users/nico_/Documents/GitHub/projetAnnuel3IBD/projet/models/"

In [14]:
pathDatasetTrain = "C:/Users/nico_/Documents/ConvertedImages/Bigdataset_Crop_Color_50x50/train/"
pathDatasetPredict = "C:/Users/nico_/Documents/ConvertedImages/Bigdataset_Crop_Color_50x50/test/"


start_time = time.time()

imagesNameTrain = os.listdir(pathDatasetTrain)
imagesNamePredict = os.listdir(pathDatasetPredict)

# Parameters
numberImageTrain = len(imagesNameTrain)
component  = 3 # color component
imageW = 50
imageH = 50
dividedPixel = 1
inputCountPerSample = (imageW * imageH * component)
gamma = 1
k = 200
it = 10
start_time = time.time()
selectedImagesTrain = random.sample(imagesNameTrain, numberImageTrain)
selectedImagesTrain = convertListToString(selectedImagesTrain, pathDatasetTrain)

pMatrixX, pMatrixY = prepareDataset(myDll, selectedImagesTrain, imageW, imageH, numberImageTrain, component, dividedPixel, -1, -1)


In [15]:
# Create & Allocate RBF Model
myDll.createRBFModel.argtypes = [ c_uint ]
myDll.createRBFModel.restype = c_void_p
pArrayWeight = myDll.createRBFModel(inputCountPerSample)

In [16]:
# Fit RBF with regression version
myDll.fitRBFRegression.argtypes = [ c_void_p, c_void_p, c_void_p, c_double, c_int, c_int ]
error = myDll.fitRBFRegression( pArrayWeight, pMatrixX, pMatrixY, gamma, k, it)

In [17]:
# Params function
myDll.predictRBFRegression.argtypes = [ c_void_p, c_void_p]
myDll.predictRBFRegression.restype = c_double

In [18]:
numberImagePredict = len (imagesNamePredict)
selectedImagesPredict = random.sample(imagesNamePredict, numberImagePredict)
selectedImagesPredict = [pathDatasetPredict + el for el in selectedImagesPredict ]

average = 0

for image in selectedImagesPredict:
    imageName = image[image.rfind("/")+1:]
    age = int(imageName[:imageName.find("_")])

    pMatrixXPredict, pMatrixYPredict = prepareDataset(myDll, image, imageW, imageH, 1, component, 1, -1, -1)
    pVectorXPredict = matrixToVector(myDll, pMatrixXPredict, imageW * imageH * component, 0)

    res = myDll.predictRBFRegression( pArrayWeight, pVectorXPredict )

    deleteDatasetMatrix(myDll,  pMatrixXPredict, pMatrixYPredict)

    #print("age : ", age, "/ predicted : ", res)
    average += (round(res) - age)**2
average =  average / len(selectedImagesPredict)
print("Moyenne erreurs Train: %s" % average**0.5 )
print("--- %s seconds ---" % (time.time() - start_time))



Moyenne erreurs Train: 38.43898126553473
--- 4468.582612752914 seconds ---
